<a href="https://colab.research.google.com/github/lolonao/code-playground/blob/main/Amazon%E3%81%8B%E3%82%89%E5%95%86%E5%93%81%E8%A9%B3%E7%B4%B0%E6%83%85%E5%A0%B1%E3%82%92%E5%8F%96%E5%BE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%shell

cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF


apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg


cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.z6MgJZgurh/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.4jQyHdIR2O/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.umwtCtWpcv/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [3]:
!apt-get update
!apt-get install chromium chromium-driver
!pip install selenium

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:4 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:10 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:12 http://deb.debian.org/debian buster-updates/main amd64 Packages [9,745 B]
Get:13 http://deb.debian.org/debian-security buster/updates/main amd64 Packages [602 kB]
Hit:14 ht

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
import io
from PIL import Image
from urllib.request import urlopen as uReq

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(10)


In [5]:
import os
from google.colab import drive
drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive/")

Mounted at /content/drive


In [ ]:
!pip install requests==2.28.2 bs4==0.0.1 beautifulsoup4

In [10]:
"""
とりあえず１件だけ商品情報を表示
"""
import requests
from bs4 import BeautifulSoup
import time

# url = 'https://www.amazon.co.jp/Pearl-HB-2649-inches-Tsubamesanjo-Racking/dp/B074MYN3BX/ref=sr_1_1_sspa?crid=18CIBDBUS6B4L&keywords=%E3%83%91%E3%83%BC%E3%83%AB%E9%87%91%E5%B1%9E%2B%E6%97%A5%E6%9C%AC%E8%A3%BD&qid=1679988716&rnid=3428066051&rps=1&s=home&sprefix=%E3%83%91%E3%83%BC%E3%83%AB%E9%87%91%E5%B1%9E%2Bmade%2Bin%2Bjapan%2Ckitchen%2C159&sr=1-1-spons&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUFHTFhWQ0tQMUM5RE8mZW5jcnlwdGVkSWQ9QTA5MDAyMjcyRkRCQTFKRktZNkdMJmVuY3J5cHRlZEFkSWQ9QTVBM0NXTzFVRDkyRyZ3aWRnZXROYW1lPXNwX2F0ZiZhY3Rpb249Y2xpY2tSZWRpcmVjdCZkb05vdExvZ0NsaWNrPXRydWU&th=1'
url = 'https://www.amazon.co.jp/-/en/Pearl-HB-1637-Stainless-Strainer-inches/dp/B0132L58JQ/ref=sr_1_1?crid=18CIBDBUS6B4L&keywords=%E3%83%91%E3%83%BC%E3%83%AB%E9%87%91%E5%B1%9E+%E6%97%A5%E6%9C%AC%E8%A3%BD&qid=1680419623&rnid=3428066051&rps=1&s=home&sprefix=%E3%83%91%E3%83%BC%E3%83%AB%E9%87%91%E5%B1%9E+made+in+japan%2Ckitchen%2C159&sr=1-1'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

time.sleep(3)
print("\n開始")

try:
    # ブランド名の取得
    brand = soup.find('a', {'id': 'bylineInfo'}).text.strip()
    print(f"Brand: {brand}")
except Exception as e:
    pass

# 商品名の取得
title = soup.find('span', {'id': 'productTitle'}).text.strip()
print(f"Title: {title}")

# ASINの取得
asin = soup.find('input', {'name': 'ASIN'}).get('value')
print(f"ASIN: {asin}")

# 商品説明の取得
description = soup.find('div', {'id': 'productDescription'}).text.strip()
print(f"Description: {description}")

# 製品仕様の取得
specifications_table = soup.find('table', {'id': 'productDetails_techSpec_section_1'})
specifications = {}
for row in specifications_table.find_all('tr'):
    cells = row.find_all('td')
    if len(cells) == 2:
        key = cells[0].text.strip()
        value = cells[1].text.strip()
        specifications[key] = value

print("Specifications:")
for key, value in specifications.items():
    print(f"{key}: {value}")




開始


AttributeError: ignored

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

# from selenium import webdriver
# from selenium.webdriver.common.by import By
import time
import requests
import io
from PIL import Image
from urllib.request import urlopen as uReq

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(10)

try:
    # NG
    url = 'https://www.amazon.co.jp/-/en/Pearl-HB-1637-Stainless-Strainer-inches/dp/B0132L58JQ/ref=sr_1_1?crid=18CIBDBUS6B4L&keywords=%E3%83%91%E3%83%BC%E3%83%AB%E9%87%91%E5%B1%9E+%E6%97%A5%E6%9C%AC%E8%A3%BD&qid=1680419623&rnid=3428066051&rps=1&s=home&sprefix=%E3%83%91%E3%83%BC%E3%83%AB%E9%87%91%E5%B1%9E+made+in+japan%2Ckitchen%2C159&sr=1-1'
    # OK
    # url = 'https://www.amazon.co.jp/Pearl-HB-2649-inches-Tsubamesanjo-Racking/dp/B074MYN3BX/ref=sr_1_1_sspa?crid=18CIBDBUS6B4L&keywords=%E3%83%91%E3%83%BC%E3%83%AB%E9%87%91%E5%B1%9E%2B%E6%97%A5%E6%9C%AC%E8%A3%BD&qid=1679988716&rnid=3428066051&rps=1&s=home&sprefix=%E3%83%91%E3%83%BC%E3%83%AB%E9%87%91%E5%B1%9E%2Bmade%2Bin%2Bjapan%2Ckitchen%2C159&sr=1-1-spons&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUFHTFhWQ0tQMUM5RE8mZW5jcnlwdGVkSWQ9QTA5MDAyMjcyRkRCQTFKRktZNkdMJmVuY3J5cHRlZEFkSWQ9QTVBM0NXTzFVRDkyRyZ3aWRnZXROYW1lPXNwX2F0ZiZhY3Rpb249Y2xpY2tSZWRpcmVjdCZkb05vdExvZ0NsaWNrPXRydWU&th=1'
    driver.get(url)
except Exception as e:
    print(f"エラー: {e}")

print('---')
time.sleep(10)

try:
    # 画像のURLを取得
    image_urls = []
    image_elements = driver.find_elements(By.XPATH, '//*[@id="main-image-container"]/ul/li/span/span/div/img')
    for element in image_elements:
        image_urls.append(element.get_attribute('src'))
    print('Image URLs:', image_urls)
except NoSuchElementException:
    print('画像が見つかりませんでした。')

print('---')

# 商品タイトルを取得
try:
    title = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="productTitle"]')))
    title = title.text.strip()
    print('商品タイトル:', title)
except NoSuchElementException:
    print('商品タイトルが見つかりませんでした。')
    pass

print('---')

# レートを取得
try:
    rate = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="averageCustomerReviews"]')))
    rate = rate.text.strip()
    print('レート:', rate)
except NoSuchElementException:
    print('レートが見つかりませんでした。')
    pass

print('---')

# 価格を取得
try:
    xpath = '//*[@id="corePriceDisplay_desktop_feature_div"]/div[1]/span[2]/span[2]'
    # xpath = '//*[@id="corePriceDisplay_desktop_feature_div"]/div[1]/span[1]/span[2]/span[2]'
    price = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, xpath)))
    price = price.text.strip()
    print('価格:', price)
except NoSuchElementException:
    print('価格が見つかりませんでした。')
    pass
except TimeoutException:
    pass

print('---')

# サイズを取得
try:
    size = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="variation_size_name"]/div/span')))
    size = size.text.strip()
    print('サイズ:', size)
except NoSuchElementException:
    print('サイズが見つかりませんでした。')
    pass

print('---')

# ブランドを取得
try:
    brand = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="productOverview_feature_div"]/div/table/tbody/tr[1]/td[2]/span')))
    brand = brand.text.strip()
    print('Brand:', brand)
except NoSuchElementException:
    print('ブランドが見つかりませんでした。')
    pass

print('---')

# 素材を取得
try:
    material = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="productOverview_feature_div"]/div/table/tbody/tr[2]/td[2]/span')))
    material = material.text.strip()
    print('Material:', material)
except NoSuchElementException:
    print('素材が見つかりませんでした。')
    pass

print('---')

# 色を取得
try:
    color = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="productOverview_feature_div"]/div/table/tbody/tr[3]/td[2]/span')))
    color = color.text.strip()
    print('Color:', color)
except NoSuchElementException:
    print('色が見つかりませんでした。')
    pass

print('---')

# 取り扱い方法を取得
try:
    care = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="productOverview_feature_div"]/div/table/tbody/tr[4]/td[2]/span')))
    care = care.text.strip()
    print('Care Instructions:', care)
except NoSuchElementException:
    print('取り扱い方法が見つかりませんでした。')
    pass

print('---')

# Handle Materialを取得
try:
    handle_material = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="productOverview_feature_div"]/div/table/tbody/tr[5]/td[2]/span')))
    handle_material = handle_material.text.strip()
    print('Handle Material:', handle_material)
except NoSuchElementException:
    print('Handle Materialが見つかりませんでした。')
    pass

print('---')

# About this itemを取得
try:
    feature_bullets = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="feature-bullets"]')))
    feature_bullets_list = feature_bullets.find_elements(By.XPATH, './ul/li/span')
    feature_bullets_text = [bullet.text.strip() for bullet in feature_bullets_list]
    print('About this item:', feature_bullets_text)
except NoSuchElementException:
    print('About this itemが見つかりませんでした。')
    pass

print('---')

# Customer ratings by featureを取得
try:
    customer_ratings = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="customerReviewsAttribute_feature_div"]/div')))
    customer_ratings_list = customer_ratings.find_elements(By.XPATH, './div[contains(@class,"a-row")]/span')
    customer_ratings_text = [rating.text.strip() for rating in customer_ratings_list]
    print('Customer ratings by feature:', customer_ratings_text)
except NoSuchElementException:
    print('Customer ratings by featureが見つかりませんでした。')
    pass
except TimeoutException:
    pass
    
print('---')

# Product Informationを取得
try:
    product_information = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="prodDetails"]')))
    product_information_list = product_information.find_elements(By.XPATH, './div[contains(@class,"a-row")]/div[contains(@class,"content")]/ul/li')
    product_information_text = [info.text.strip() for info in product_information_list]
    print('Product Information:', product_information_text)
except NoSuchElementException:
    print('Product Informationが見つかりませんでした。')
    pass

print('---')

# Product descriptionを取得
try:
    product_description = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="productDescription_feature_div"]')))
    product_description = product_description.text.strip()
    print('Product description:', product_description)
except NoSuchElementException:
    print('Product descriptionが見つかりませんでした。')
    pass

print('---')

# From the Manufactureを取得
try:
    from_the_manufacture = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="aplus"]/div')))
    from_the_manufacture_list = from_the_manufacture.find_elements(By.XPATH, './div[contains(@class,"a-row")]/div[contains(@class,"content")]/ul/li')
    from_the_manufacture_text = [info.text.strip() for info in from_the_manufacture_list]
    print('From the Manufacture:', from_the_manufacture_text)
except NoSuchElementException:
    print('From the Manufactureが見つかりませんでした。')
    pass
except TimeoutException:
    pass

######################

print('######################')

try:
    # Care Instructions
    care_instructions = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="productOverview_feature_div"]/div/table/tbody/tr[4]/td[2]/span')))
    care_instructions_text = care_instructions.text.strip()
    print('Care Instructions:', care_instructions_text)
except NoSuchElementException:
    print('Care Instructionsが見つかりませんでした。')
    pass

print('######################')
try:
    # Technical Details
    technical_details = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="prodDetails"]/div/div[1]/div[1]/div/div[2]/div/div')))
    technical_details_text = technical_details.text.strip()
    print('Technical Details:', technical_details_text)
except NoSuchElementException:
    print('Technical Detailsが見つかりませんでした。')
    pass

print('######################')
try:
    # Additional Information
    additional_information = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="productDetails_db_sections"]')))
    additional_information_text = additional_information.text.strip()
    print('Additional Information:', additional_information_text)
except NoSuchElementException:
    print('Additional Informationが見つかりませんでした。')
    pass

print('######################')
try:
    # From the Manufacture1
    from_manufacture1 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="aplus"]/div/div[1]/div/div[1]')))
    from_manufacture1_text = from_manufacture1.text.strip()
    print('From the Manufacture1:', from_manufacture1_text)
except NoSuchElementException:
    print('From the Manufacture1が見つかりませんでした。')
    pass

print('######################')
try:
    # From the Manufacture2
    from_manufacture2 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="aplus"]/div/div[2]')))
    from_manufacture2_text = from_manufacture2.text.strip()
    print('From the Manufacture2:', from_manufacture2_text)
except NoSuchElementException:
    print('From the Manufacture2が見つかりませんでした。')
    pass

# 画像を取得
# try:
#     image = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ivLargeImage"]/img')))
#     print("画像:", image)
#     image_url = image.get_attribute('src')
#     print('Image URL:', image_url)
# except NoSuchElementException:
#     print('画像が見つかりませんでした。')

print('######################')


# 画像を取得
try:
    wait = WebDriverWait(driver=driver, timeout=60)
    # 複数画像取得
    images_btn = driver.find_elements(By.CSS_SELECTOR, "li.a-spacing-small.item.imageThumbnail.a-declarative > span > span > span > input")
    for index, image_btn in enumerate(images_btn, start=1):
        # input要素を一つずつクリック
        image_btn.click()
        # time.sleep(10)
        wait.until(EC.presence_of_all_elements_located)
        try:
            img = driver.find_element(By.XPATH, f'(//div[@class="imgTagWrapper"]/img)[{index}]').get_attribute("src")
            print("[INFO]  img :", img)
        except NoSuchElementException:
            pass
except TimeoutException:
    print('画像が見つかりませんでした。')
    pass


def resize_amazon_image_url(url: str, size: str) -> str:
    # URLからファイル名を抽出する
    filename_start = url.rfind('/')
    filename = url[filename_start+1:]

    # URLからファイル名以外の部分を抽出する
    url_prefix = url[:filename_start+1]
    url_suffix = url[url.find('.jpg') + 4:]

    # サイズを変更する
    size_str = f"_{size}." if size else "."
    new_filename = filename.replace('SX450', f"SL{size_str}")
    
    # 新しいURLを作成する
    new_url = url_prefix + new_filename + url_suffix
    return new_url

url = 'https://m.media-amazon.com/images/I/61lg0bA0w+L._AC_SX450_.jpg'
new_url = resize_amazon_image_url(url, '1000')
print(new_url)

---
Image URLs: ['https://m.media-amazon.com/images/I/81duFeYtjvL._AC_SX425_.jpg']
---
商品タイトル: Pearl Metal HB-1637 Stainless Steel Strainer, 5.9 inches (15 cm), Made in Japan
---
レート: 3 ratings
---
---
サイズ: 15cm
---
Brand: Stainless Steel
---
Material: multi
---
Color: パール金属(PEARL METAL)
---
Care Instructions: ざる
---
Handle Material: 110 Grams
---
About this item: ['Product Size (Approx.): Outer Diameter 6.5 x Height 2.8 inches (165 x 7', 'Product Weight: Approx. 4.3 oz (110 g)', 'Material: 18-8 Stainless Steel', 'Made in Japan']
---
---
Product Information: []
---
Product description: Product description
Made in Tsubame Sanjo, Niigata Prefecture, Japan.
Made of durable and hygienic stainless steel
Comes with feet so it does not touch the sink directly and is easy to drain.
---
######################
Care Instructions: ざる
######################
Technical Details: Manufacturer パール金属(PEARL METAL)
Model Number HB-1637
Package Dimensions 17.9 x 16.4 x 10.2 cm
Material Stainless Steel
Color

TimeoutException: ignored

In [17]:
# import io
import time
# import requests
from selenium import webdriver
from selenium.webdriver.common import timeouts
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

# from PIL import Image
# from urllib.request import urlopen as uReq

TIMEOUT_SEC = int(10)

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(TIMEOUT_SEC)


try:
    # NG ザル
    url = 'https://www.amazon.co.jp/-/en/Pearl-HB-1637-Stainless-Strainer-inches/dp/B0132L58JQ/ref=sr_1_1?crid=18CIBDBUS6B4L&keywords=%E3%83%91%E3%83%BC%E3%83%AB%E9%87%91%E5%B1%9E+%E6%97%A5%E6%9C%AC%E8%A3%BD&qid=1680419623&rnid=3428066051&rps=1&s=home&sprefix=%E3%83%91%E3%83%BC%E3%83%AB%E9%87%91%E5%B1%9E+made+in+japan%2Ckitchen%2C159&sr=1-1'
    # OK なべ
    # url = 'https://www.amazon.co.jp/Pearl-HB-2649-inches-Tsubamesanjo-Racking/dp/B074MYN3BX/ref=sr_1_1_sspa?crid=18CIBDBUS6B4L&keywords=%E3%83%91%E3%83%BC%E3%83%AB%E9%87%91%E5%B1%9E%2B%E6%97%A5%E6%9C%AC%E8%A3%BD&qid=1679988716&rnid=3428066051&rps=1&s=home&sprefix=%E3%83%91%E3%83%BC%E3%83%AB%E9%87%91%E5%B1%9E%2Bmade%2Bin%2Bjapan%2Ckitchen%2C159&sr=1-1-spons&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUFHTFhWQ0tQMUM5RE8mZW5jcnlwdGVkSWQ9QTA5MDAyMjcyRkRCQTFKRktZNkdMJmVuY3J5cHRlZEFkSWQ9QTVBM0NXTzFVRDkyRyZ3aWRnZXROYW1lPXNwX2F0ZiZhY3Rpb249Y2xpY2tSZWRpcmVjdCZkb05vdExvZ0NsaWNrPXRydWU&th=1'
    driver.get(url)
except Exception as e:
    print(f"エラー: {e}")

wait = WebDriverWait(driver=driver, timeout=TIMEOUT_SEC)

print('---')

try:
    # 画像のURLを取得
    image_urls = []
    xpath = '//*[@id="main-image-container"]/ul/li/span/span/div/img'
    image_elements = driver.find_elements(By.XPATH, xpath)
    for element in image_elements:
        image_urls.append(element.get_attribute('src'))
    print('Image URLs:', image_urls)
except NoSuchElementException:
    print('画像が見つかりませんでした。')

print('---')

# 商品タイトルを取得
try:
    xpath = '//*[@id="productTitle"]'
    wait = WebDriverWait(driver=driver, timeout=TIMEOUT_SEC)
    title = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
    title = title.text.strip()
    print('商品タイトル:', title)
except NoSuchElementException:
    print('商品タイトルが見つかりませんでした。')
    pass

print('---')

# レートを取得
try:
    xpath = '//*[@id="averageCustomerReviews"]'
    rate = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
    rate = rate.text.strip()
    print('レート:', rate)
except NoSuchElementException:
    print('レートが見つかりませんでした。')
    pass

print('---')

# 価格を取得
try:
    # xpath = '//*[@id="corePriceDisplay_desktop_feature_div"]/div[1]/span[2]/span[2]'
    # xpath = '//*[@id="corePriceDisplay_desktop_feature_div"]/div[1]/span[1]/span[2]/span[2]'
    # xpath = '//*[@id="corePriceDisplay_desktop_feature_div"]/div[1]/span[1]/span[2]/span[2]'
    xpath = '//*[@id="corePriceDisplay_desktop_feature_div"]/div[1]'
    price_element = driver.find_element(By.XPATH, xpath)
    print(f"price_element: {price_element}")
    price = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
    price = price.text.strip()
    print('価格:', price)
except NoSuchElementException:
    print('価格が見つかりませんでした。')
    pass
except TimeoutException:
    pass

print('---')

# サイズを取得
try:
    size = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="variation_size_name"]/div/span')))
    size = size.text.strip()
    print('サイズ:', size)
except NoSuchElementException:
    print('サイズが見つかりませんでした。')
    pass

print('---')

# ブランドを取得
try:
    brand = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="productOverview_feature_div"]/div/table/tbody/tr[1]/td[2]/span')))
    brand = brand.text.strip()
    print('Brand:', brand)
except NoSuchElementException:
    print('ブランドが見つかりませんでした。')
    pass

print('---')

# 素材を取得
try:
    material = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="productOverview_feature_div"]/div/table/tbody/tr[2]/td[2]/span')))
    material = material.text.strip()
    print('Material:', material)
except NoSuchElementException:
    print('素材が見つかりませんでした。')
    pass

print('---')

# 色を取得
try:
    color = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="productOverview_feature_div"]/div/table/tbody/tr[3]/td[2]/span')))
    color = color.text.strip()
    print('Color:', color)
except NoSuchElementException:
    print('色が見つかりませんでした。')
    pass

print('---')

# 取り扱い方法を取得
try:
    care = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="productOverview_feature_div"]/div/table/tbody/tr[4]/td[2]/span')))
    care = care.text.strip()
    print('Care Instructions:', care)
except NoSuchElementException:
    print('取り扱い方法が見つかりませんでした。')
    pass

print('---')

# Handle Materialを取得
try:
    handle_material = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="productOverview_feature_div"]/div/table/tbody/tr[5]/td[2]/span')))
    handle_material = handle_material.text.strip()
    print('Handle Material:', handle_material)
except NoSuchElementException:
    print('Handle Materialが見つかりませんでした。')
    pass

print('---')

# About this itemを取得
try:
    feature_bullets = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="feature-bullets"]')))
    feature_bullets_list = feature_bullets.find_elements(By.XPATH, './ul/li/span')
    feature_bullets_text = [bullet.text.strip() for bullet in feature_bullets_list]
    print('About this item:', feature_bullets_text)
except NoSuchElementException:
    print('About this itemが見つかりませんでした。')
    pass

print('---')

# Customer ratings by featureを取得
try:
    customer_ratings = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="customerReviewsAttribute_feature_div"]/div')))
    customer_ratings_list = customer_ratings.find_elements(By.XPATH, './div[contains(@class,"a-row")]/span')
    customer_ratings_text = [rating.text.strip() for rating in customer_ratings_list]
    print('Customer ratings by feature:', customer_ratings_text)
except NoSuchElementException:
    print('Customer ratings by featureが見つかりませんでした。')
    pass
except TimeoutException:
    pass
    
print('---')

# Product Informationを取得
try:
    product_information = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="prodDetails"]')))
    product_information_list = product_information.find_elements(By.XPATH, './div[contains(@class,"a-row")]/div[contains(@class,"content")]/ul/li')
    product_information_text = [info.text.strip() for info in product_information_list]
    print('Product Information:', product_information_text)
except NoSuchElementException:
    print('Product Informationが見つかりませんでした。')
    pass

print('---')

# Product descriptionを取得
try:
    product_description = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="productDescription_feature_div"]')))
    product_description = product_description.text.strip()
    print('Product description:', product_description)
except NoSuchElementException:
    print('Product descriptionが見つかりませんでした。')
    pass

print('---')

# From the Manufactureを取得
try:
    from_the_manufacture = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="aplus"]/div')))
    from_the_manufacture_list = from_the_manufacture.find_elements(By.XPATH, './div[contains(@class,"a-row")]/div[contains(@class,"content")]/ul/li')
    from_the_manufacture_text = [info.text.strip() for info in from_the_manufacture_list]
    print('From the Manufacture:', from_the_manufacture_text)
except NoSuchElementException:
    print('From the Manufactureが見つかりませんでした。')
    pass
except TimeoutException:
    pass

######################

print('######################')

try:
    # Care Instructions
    xpath: str = '//*[@id="productOverview_feature_div"]/div/table/tbody/tr[4]/td[2]/span'
    care_instructions = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
    care_instructions_text = care_instructions.text.strip()
    print('Care Instructions:', care_instructions_text)
except NoSuchElementException:
    print('Care Instructionsが見つかりませんでした。')
    pass

print('######################')
try:
    # Technical Details
    xpath: str = '//*[@id="prodDetails"]/div/div[1]/div[1]/div/div[2]/div/div'
    technical_details = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
    technical_details_text = technical_details.text.strip()
    print('Technical Details:', technical_details_text)
except NoSuchElementException:
    print('Technical Detailsが見つかりませんでした。')
    pass
except TimeoutException:
    print("タイムアウト")
    pass

print('######################')
try:
    # Additional Information
    xpath: str = '//*[@id="productDetails_db_sections"]'
    additional_information = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
    additional_information_text = additional_information.text.strip()
    print('Additional Information:', additional_information_text)
except NoSuchElementException:
    print('Additional Informationが見つかりませんでした。')
    pass
except TimeoutException:
    print("タイムアウト")
    pass

print('######################')
try:
    # From the Manufacture1
    xpath: str = '//*[@id="aplus"]/div/div[1]/div/div[1]'
    manufacture1 = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
    manufacture1_text = manufacture1.text.strip()
    print('From the Manufacture1:', manufacture1_text)
except NoSuchElementException:
    print('From the Manufacture1が見つかりませんでした。')
    pass
except TimeoutException:
    print("タイムアウト")
    pass

print('######################')
try:
    # From the Manufacture2
    from_manufacture2 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="aplus"]/div/div[2]')))
    from_manufacture2_text = from_manufacture2.text.strip()
    print('From the Manufacture2:', from_manufacture2_text)
except NoSuchElementException:
    print('From the Manufacture2が見つかりませんでした。')
    pass
except TimeoutException:
    print("タイムアウト")
    pass

# 画像を取得
# try:
#     image = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ivLargeImage"]/img')))
#     print("画像:", image)
#     image_url = image.get_attribute('src')
#     print('Image URL:', image_url)
# except NoSuchElementException:
#     print('画像が見つかりませんでした。')

print('######################')


# 画像を取得
try:
    wait = WebDriverWait(driver=driver, timeout=60)
    # 複数画像取得
    images_btn = driver.find_elements(By.CSS_SELECTOR, "li.a-spacing-small.item.imageThumbnail.a-declarative > span > span > span > input")
    for index, image_btn in enumerate(images_btn, start=1):
        # input要素を一つずつクリック
        image_btn.click()
        # time.sleep(10)
        wait.until(EC.presence_of_all_elements_located)
        try:
            img = driver.find_element(By.XPATH, f'(//div[@class="imgTagWrapper"]/img)[{index}]').get_attribute("src")
            print("[INFO]  img :", img)
        except NoSuchElementException:
            pass
except TimeoutException:
    print('画像が見つかりませんでした。')
    pass


def resize_amazon_image_url(url: str, size: str) -> str:
    # URLからファイル名を抽出する
    filename_start = url.rfind('/')
    filename = url[filename_start+1:]

    # URLからファイル名以外の部分を抽出する
    url_prefix = url[:filename_start+1]
    url_suffix = url[url.find('.jpg') + 4:]

    # サイズを変更する
    size_str = f"_{size}." if size else "."
    new_filename = filename.replace('SX450', f"SL{size_str}")

    # 新しいURLを作成する
    new_url = url_prefix + new_filename + url_suffix
    return new_url

print("大きいサイズの画像URLに変換")
url = 'https://m.media-amazon.com/images/I/61lg0bA0w+L._AC_SX450_.jpg'
new_url = resize_amazon_image_url(url, '1000')
print(new_url)

print('######################')
try:
    # From 価格テーブル
    xpath = '//*[@id="variation_size_name"]'
    from_manufacture2 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
    from_manufacture2_text = from_manufacture2.text.strip()
    print('From 価格テーブル:', from_manufacture2_text)
except NoSuchElementException:
    print('価格テーブルが見つかりませんでした。')
    pass
except TimeoutException:
    print("タイムアウト")
    pass




---
Image URLs: ['https://m.media-amazon.com/images/I/81duFeYtjvL._AC_SX425_.jpg']
---
商品タイトル: Pearl Metal HB-1637 Stainless Steel Strainer, 5.9 inches (15 cm), Made in Japan
---
レート: 3 ratings
---
価格が見つかりませんでした。
---
サイズ: 15cm
---
Brand: Stainless Steel
---
Material: multi
---
Color: パール金属(PEARL METAL)
---
Care Instructions: ざる
---
Handle Material: 110 Grams
---
About this item: ['Product Size (Approx.): Outer Diameter 6.5 x Height 2.8 inches (165 x 7', 'Product Weight: Approx. 4.3 oz (110 g)', 'Material: 18-8 Stainless Steel', 'Made in Japan']
---
---
Product Information: []
---
Product description: Product description
Made in Tsubame Sanjo, Niigata Prefecture, Japan.
Made of durable and hygienic stainless steel
Comes with feet so it does not touch the sink directly and is easy to drain.
---
######################
Care Instructions: ざる
######################
Technical Details: Manufacturer パール金属(PEARL METAL)
Model Number HB-1637
Package Dimensions 17.9 x 16.4 x 10.2 cm
Material Stainl